# Visualizar terremotos

Estos datos sobre terremotos son cortesía de [USGS](https://earthquake.usgs.gov).

{query_address}

Escribe una dirección para ordenar los terremotos recientes por distancia.

In [1]:
target_folder = '/tmp'
query_address = 'David, Chiriquí'

In [2]:
from geopy.geocoders import GoogleV3
g = GoogleV3()
query_location = g.geocode(query_address)
query_point = query_location.point
query_point

Point(8.4007278, -82.4427769, 0.0)

In [3]:
import pandas as pd

In [4]:
table_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.csv'
table = pd.read_csv(table_url, parse_dates=['time'])
table.ix[0]

time               2017-03-14 05:06:54.460000
latitude                              34.0388
longitude                            -117.239
depth                                   17.64
mag                                      3.61
magType                                    mw
nst                                       182
gap                                        15
dmin                                  0.01436
rms                                       0.2
net                                        ci
id                                 ci37828544
updated              2017-03-16T14:49:50.521Z
place               2km ESE of Loma Linda, CA
type                               earthquake
horizontalError                          0.11
depthError                               0.22
magError                                  NaN
magNst                                      4
status                               reviewed
locationSource                             ci
magSource                         

In [5]:
table = table[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
table.ix[0]

time         2017-03-14 05:06:54.460000
latitude                        34.0388
longitude                      -117.239
depth                             17.64
mag                                3.61
place         2km ESE of Loma Linda, CA
Name: 0, dtype: object

In [6]:
from geopy.distance import vincenty as get_distance

def get_distance_in_km(x):
    return pd.Series(get_distance(query_point, (x.latitude, x.longitude)).kilometers)
    
table['distanceInKm'] = table.apply(get_distance_in_km, axis=1)
table = table.sort_values(['distanceInKm'])
table[:3]

,time,latitude,longitude,depth,mag,place,distanceInKm
5,2017-02-21 14:09:04.320,-19.281400,-63.904700,595.98,6.50,"41km E of Padilla, Bolivia",3675.661822
6,2017-02-18 12:10:17.970,-23.861300,-66.659200,222.00,6.30,"52km NW of San Antonio de los Cobres, Argentina",3960.251187
0,2017-03-14 05:06:54.460,34.038833,-117.238667,17.64,3.61,"2km ESE of Loma Linda, CA",4553.545489


In [7]:
from datetime import datetime
now = datetime.utcnow()

def get_elapsed_time_in_negative_seconds(x):
    return -1 * (now - x).seconds

geotable = table.copy()
geotable['fillReds'] = geotable['time'].apply(get_elapsed_time_in_negative_seconds)
geotable['radiusInPixelsRange3-27'] = geotable['mag']
geotable.ix[0]

time                       2017-03-14 05:06:54.460000
latitude                                      34.0388
longitude                                    -117.239
depth                                           17.64
mag                                              3.61
place                       2km ESE of Loma Linda, CA
distanceInKm                                  4553.55
fillReds                                       -46369
radiusInPixelsRange3-27                          3.61
Name: 0, dtype: object

In [8]:
from os.path import join

table_path = join(target_folder, 'terremotos.csv')
table.to_csv(table_path, index=False)
print('terremotos_table_path = %s' % table_path)

geotable_path = join(target_folder, 'terremotos-mapa.csv')
geotable.to_csv(geotable_path, index=False)
print('terremotos_satellite_geotable_path = %s' % geotable_path)

terremotos_table_path = /tmp/terremotos.csv
terremotos_satellite_geotable_path = /tmp/terremotos-mapa.csv


# Terremotos visualizados

{terremotos_table}

{terremotos_satellite_geotable}

El radio del círculo indica la magnitud.

El color del círculo indica el tiempo.

[USGS](https://earthquake.usgs.gov) ofrece más tablas sobre terremotos recientes.